<a href="https://colab.research.google.com/github/hushuangwei/rgn2-replica/blob/main/rgn2_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* How would coevolution implicitly affect the language model training?
* Would kNN-style unsupervised learning useful for RGN2?

In [35]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [ ]:
!git clone https://github.com/hushuangwei/rgn2-replica.git

In [2]:
# https://blog.csdn.net/NEUdeep/article/details/115724826
!export PYTHONWARNINGS='ignore:semaphore_tracker:UserWarning'

In [ ]:
!pip install wandb sidechainnet einops proDy tqdm datasets transformers x-transformers pytorch-lightning fair-esm En-transformer pytorch3d invariant_point_attention

In [4]:
%cd rgn2-replica/scripts/

/content/rgn2-replica/scripts


One can skip this google drive setting and download sidechainnet data directly but more slowly

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!mkdir -p /content/rgn2-replica/scripts/sidechainnet_data/
!cp /content/drive/MyDrive/protein/sidechainnet_casp12_90.pkl /content/rgn2-replica/scripts/sidechainnet_data/

In [7]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [33]:
!nohup python train_rgn2.py --device cuda:0 --wb_entity hushuangwei \
   --wb_proj rgn2_replica --run_name RGN2_ipa_1e-4 \
   --min_len_valid 50 --casp_version 12 --scn_thinning 90 \
   --min_len 0 --max_len 384 --input_dropout 0.1 --num_layers 6 \
   --bidirectional 1 --layer_type LSTM --act aconc --num_recycles_train 8 \
   --refiner_args "{\"refiner_type\": \"IPA\"}" \
   > RGN2X_vanillaLSTM_full_run_logs.txt 2>&1 &

In [32]:
!tail -f RGN2X_vanillaLSTM_full_run_logs.txt

wandb:            loss 0.08731
wandb:            rmsd 9.9361
wandb:    torsion_loss 1.76916
wandb:       viol_loss 0.00342
wandb: 
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Synced RGN2_ipa_1e-4: https://wandb.ai/hushuangwei/rgn2_replica/runs/372ib897
wandb: Find logs at: ./wandb/run-20211010_110523-372ib897/logs/debug.log
wandb: 

^C


In [27]:
!pkill -f train_rgn2.py

In [61]:
!ps aux | grep train_rgn2.py

root        2285  0.0  0.0  39200  6252 ?        S    12:47   0:00 /bin/bash -c ps aux | grep train_rgn2.py
root        2287  0.0  0.0  38572  5180 ?        S    12:47   0:00 grep train_rgn2.py


In [53]:
!nvidia-smi

Tue Oct  5 08:13:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0   103W / 300W |  11293MiB / 16160MiB |     59%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# below is unfinished

In [56]:
import sidechainnet as scn
import random 

import sys
sys.path.append("..")

import torch
import py3Dmol
import esm

from rgn2_replica import *
from rgn2_replica.rgn2 import *
from rgn2_replica.rgn2_utils import *
from rgn2_replica.rgn2_trainers import *

In [78]:
from sidechainnet.utils.sequence import ProteinVocabulary as VOCAB
VOCAB = VOCAB()

In [81]:
set_seed(42)
dataloaders = scn.load(casp_version=12, thinning=90, with_pytorch="dataloaders", 
                                batch_size=1, dynamic_batching=False)

SidechainNet was loaded from ./sidechainnet_data/sidechainnet_casp12_90.pkl.


In [82]:
# if torch.cuda.is_available():
#     device = torch.device("cuda")
# else:
#     device = torch.device("cpu")
device = "cpu"
model =  RGN2_IPA(embedding_dim=1284).to(device)

In [83]:
save_path = "/content/rgn2-replica/scripts/rgn2_models/RGN2_ipa_1e-4@_32K.pt"
model.load_state_dict(torch.load(save_path))
sum([p.numel() for p in model.parameters()])

34216661

In [84]:
dataloaders["train"].dataset
MIN_LEN_TEST = 70
MIN_LEN = 0
MAX_LEN = 512

In [85]:
embedder, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()

In [86]:
embedder = embedder.to(device)

In [90]:
### TEST
tic = time.time()
get_prot_test_ = mp_nerf.utils.get_prot( 
    dataloader_=dataloaders, 
    vocab_=VOCAB, # mp_nerf.utils.
    min_len=MIN_LEN, max_len=MAX_LEN, 
    verbose=False, subset="test"
)
# get num of unique, full-masked proteins
seqs = []
for i, prot_args in enumerate(dataloaders["test"].dataset):
    # (id, int_seq, mask, ... , str_seq)
    length = len(prot_args[-1]) 
    if 0 < length < MAX_LEN and sum( prot_args[2] ) == length:
        seqs.append( prot_args[-1] )

metrics_stuff_test = predict(
    get_prot_= get_prot_test_, 
    steps = len(set(seqs)), # 24 for MIN_LEN=70
    model = model,
    embedder = embedder, 
    return_preds = True,
    log_every = 4,
    accumulate_every = len(set(seqs)),
    seed = 42, # 42
    mode = "fast_test", # "test" # "test" is for AR, "fast_test" is for iterative
    recycle_func = lambda x: 1, # 5 # 3 # 2 
    wandbai = False,
)
preds_list_test, metrics_list_test, metrics_stats_test = metrics_stuff_test
print("\n", "Test Results:", sep="")
for k,v in metrics_stats_test.items():
    offset = " " * ( max(len(ki) for ki in metrics_stats_test.keys()) - len(k) )
    print(k + offset, ":", v)
print("\n")
print("Time taken: ", time.time()-tic, "\n")

torch.Size([3, 246, 2])


TypeError: ignored